In [53]:
import numpy as np
import pandas as pd

In [54]:
df = pd.read_csv('card_cust.csv')

In [55]:
df['MINIMUM_PAYMENTS']=df['MINIMUM_PAYMENTS'].fillna(df['MINIMUM_PAYMENTS'].mean())

### Q1

For Vertorization vs Groupby (pivot_table)

In [56]:
# For Verterization

df1 = df.copy()
t_list = df1['TENURE'].unique()
t_list

array([12.,  8., 11.,  9., 10.,  7.,  6.])

In [57]:
# for 구문확인
TENURE_i = df1['TENURE']==12
df1.loc[TENURE_i,['CREDIT_LIMIT','BALANCE']].corr().loc['CREDIT_LIMIT','BALANCE']

0.4608334883447319

In [66]:
corr_list = []
for i in t_list:
    TENURE_i = df1['TENURE']==i    # df1.query(" ...  ") 상수만 사용
    corr_list.append(df1.loc[TENURE_i,['CREDIT_LIMIT','BALANCE']].corr().loc['CREDIT_LIMIT','BALANCE'])

# pd.DataFrame(corr_list, index=t_list)
pd.DataFrame( 
    {
    'TENURE':t_list,
    'corr':corr_list
    }
)

,TENURE,corr
0,12.0,0.460833
1,8.0,0.820696
2,11.0,0.380360
3,9.0,0.085474
4,10.0,0.291482
5,7.0,0.948405
6,6.0,0.868056


In [43]:
## Groupby
df1_1 = df1.copy()
my_t_group = df1_1.groupby('TENURE')[['CREDIT_LIMIT','BALANCE']].agg('corr')
my_t_group

CREDIT_LIMIT   BALANCE
TENURE                                     
6.0    CREDIT_LIMIT      1.000000  0.868056
       BALANCE           0.868056  1.000000
7.0    CREDIT_LIMIT      1.000000  0.948405
       BALANCE           0.948405  1.000000
8.0    CREDIT_LIMIT      1.000000  0.820696
       BALANCE           0.820696  1.000000
9.0    CREDIT_LIMIT      1.000000  0.085474
       BALANCE           0.085474  1.000000
10.0   CREDIT_LIMIT      1.000000  0.291482
       BALANCE           0.291482  1.000000
11.0   CREDIT_LIMIT      1.000000  0.380360
       BALANCE           0.380360  1.000000
12.0   CREDIT_LIMIT      1.000000  0.460833
       BALANCE           0.460833  1.000000

In [68]:
my_t_group.xs(('BALANCE'),level=1)['CREDIT_LIMIT']

TENURE
6.0     0.868056
7.0     0.948405
8.0     0.820696
9.0     0.085474
10.0    0.291482
11.0    0.380360
12.0    0.460833
Name: CREDIT_LIMIT, dtype: float64

In [69]:
### pivot_table

my_t_pivot = pd.pivot_table(df1_1,index=['TENURE'],values=['CREDIT_LIMIT','BALANCE'],aggfunc='corr')
my_t_pivot

BALANCE  CREDIT_LIMIT
TENURE                                     
6.0    CREDIT_LIMIT  0.868056      1.000000
       BALANCE       1.000000      0.868056
7.0    CREDIT_LIMIT  0.948405      1.000000
       BALANCE       1.000000      0.948405
8.0    CREDIT_LIMIT  0.820696      1.000000
       BALANCE       1.000000      0.820696
9.0    CREDIT_LIMIT  0.085474      1.000000
       BALANCE       1.000000      0.085474
10.0   CREDIT_LIMIT  0.291482      1.000000
       BALANCE       1.000000      0.291482
11.0   CREDIT_LIMIT  0.380360      1.000000
       BALANCE       1.000000      0.380360
12.0   CREDIT_LIMIT  0.460833      1.000000
       BALANCE       1.000000      0.460833

In [70]:
my_t_pivot.xs('BALANCE',level=1)['CREDIT_LIMIT'].nlargest(1)

TENURE
7.0    0.948405
Name: CREDIT_LIMIT, dtype: float64

### Q2

In [46]:
df2=df.copy()
df2.columns

Index(['CUST_ID', 'BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'],
      dtype='object')

In [47]:
from sklearn.preprocessing import StandardScaler
mySDS = StandardScaler()

In [48]:
df2.loc[:,'BALANCE':'TENURE']=mySDS.fit_transform(df2.loc[:,'BALANCE':'TENURE'])

In [49]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [51]:
df2_1 =df.copy()
sil_list=[]

for k in [2,3,4,5]:
    myKM = KMeans(n_clusters=k,random_state=1234)
    myKM.fit(df2.loc[:,'BALANCE':'TENURE'])
    # 기존 df 에 군집 label column 을 삽입한다.
    df2_1[k]=myKM.labels_
    sil_list.append(silhouette_score(df2.loc[:,'BALANCE':'TENURE'],myKM.labels_))
    
pd.DataFrame(sil_list,index=[2,3,4,5], columns=['silhouette_score'])


/Users/smjune/Works/ipynb/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/smjune/Works/ipynb/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/smjune/Works/ipynb/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/smjune/Works/ipynb/.venv/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  war

,silhouette_score
2,0.307528
3,0.196361
4,0.207151
5,0.192741


In [25]:
# 최적의 k = 2 
# k=2 일대 2개 군집별 (0,1) ONEOFF_PURCHASES 의 평균

df2_1.groupby(2)['ONEOFF_PURCHASES'].mean()

2
0     340.230998
1    3946.187525
Name: ONEOFF_PURCHASES, dtype: float64

### Q3

In [26]:
df3 =df.copy()
df3.columns

Index(['CUST_ID', 'BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
       'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE'],
      dtype='object')

In [27]:
test = df3['CUST_ID']%4==0
X_list = [ 'BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
        'INSTALLMENTS_PURCHASES', 'CASH_ADVANCE',
       'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
       'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY',
       'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS',
       'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']

In [28]:
X_train = df3.loc[~test, X_list]
X_test = df3.loc[test, X_list]
y_train = df3.loc[~test,'ONEOFF_PURCHASES']
y_test= df3.loc[test,'ONEOFF_PURCHASES']

In [29]:
from sklearn.tree import DecisionTreeRegressor

In [30]:
myDTR = DecisionTreeRegressor(random_state=1234)

In [31]:
myDTR.fit(X_train, y_train)

DecisionTreeRegressor(random_state=1234)

In [32]:
y_pred = myDTR.predict(X_test)

In [33]:
from sklearn.metrics import mean_squared_error

In [34]:
mean_squared_error(y_test, y_pred, squared=False)

1039.193967231063

In [35]:
# squared = False 을 사용할 수 없을 때

mean_squared_error(y_test, y_pred)**0.5

1039.193967231063